# Send data to Arize in 5 easy steps < 5 min!

This is super quick tutorial to show you how easy it is to send data into the Arize platform.

#Step 1 - Install and Import the Arize Python SDK

In [ ]:
!pip install arize

import pandas as pd
from arize.pandas.logger import Client, Schema
from arize.utils.types import ModelTypes, Environments

#Step 2 - Grab Your Data
We've already got the data for you and broken it out into 3 pandas dataframes.

In [ ]:
train_df = pd.read_csv('https://storage.googleapis.com/arize-assets/documentation-sample-data/fraud/fraud_train.csv?raw=true', index_col=False)
prod_df = pd.read_csv("https://storage.googleapis.com/arize-assets/documentation-sample-data/fraud/fraud_production.csv?raw=true", index_col=False)


#Step 3 - Grab Your Space Key and API Key
You can find your keys in the Arize Platform on the left-hand side menu under -> "Space Settings"

<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [ ]:
SPACE_KEY = "SPACE_KEY"
API_KEY = "API_KEY"
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)
if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Import and Setup Arize Client Done! Now we can start using Arize!")

#Step 4 - Name Your Model and Set Model Version

In [ ]:
model_id = "upload-classification-data-with-arize"
model_version = "1.0"
model_type = ModelTypes.SCORE_CATEGORICAL

#Step 5 - Send Data to the Arize Platform
We've handled the schema and logging for you! Just send in data for train, validation and production

In [ ]:
# sending training data

features = [
    'MERCHANT_TYPE',
    'ENTRY_MODE',
    'STATE',
    'MEAN_AMOUNT',
    'STD_AMOUNT',
    'TX_AMOUNT',
    'VISA_RISK_SCORE',
    'MASTERCARD_RISK_SCORE',
    'AMEX_RISK_SCORE',
]

# Define a Schema() object for Arize to pick up data from the correct columns for logging
training_schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="PREDICTION",
    prediction_score_column_name="PREDICTION_SCORE",
    actual_label_column_name="ACTUAL",
    actual_score_column_name="ACTUAL_SCORE",
    feature_column_names=features
)

# Logging Training DataFrame
training_response = arize_client.log(
    dataframe=train_df,
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    environment=Environments.TRAINING,
    schema=training_schema,
)

# If successful, the server will return a status_code of 200
if training_response.status_code != 200:
    print(f"❌ logging failed with response code {training_response.status_code}, {training_response.text}")
else:
    print(f"✅ You have successfully logged training set to Arize")


In [ ]:
# send production data

production_schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="PREDICTION",
    prediction_score_column_name="PREDICTION_SCORE",
    actual_label_column_name="ACTUAL",
    actual_score_column_name="ACTUAL_SCORE",
    feature_column_names=features
)

production_response = arize_client.log(
    dataframe=prod_df,
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    environment=Environments.PRODUCTION,
    schema=production_schema,
)

if production_response.status_code != 200:
    print(f"❌ logging failed with response code {production_response.status_code}, {production_response.text}")
else:
    print(f"✅ You have successfully logged production set to Arize")

## Check Data Ingestion Information

Data will be available in the UI in about 10 minutes after it was received. If data from a new model is sent, the model will be reflected almost immediately in the Arize platform. However, you will not see data yet. To verify data has been sent correctly and is being processed, we recommend that you check our Data Ingestion tab. 

You will be able to see the predictions, actuals, and feature importances that have been sent in the last week, last day or last 30 minutes. 

An example view of the Data Ingestion tab from a model, when data is sent continuously over 30 minutes, is shown in the image below. 

<img src="https://storage.googleapis.com/arize-assets/fixtures/data-ingestion-tab.png" width="700">

#Congratulations, you've now sent your first machine learning data to the Arize platform!!

Additional:
- If you want to remove this example model from your platform, just click "Models' -> "upload-classification-data-with-arize" -> "config" -> "delete"